In [3]:
using MLDatasets: MNIST
using Flux
include("RNN_AD/rnn.jl")

train_data = MNIST(:train)
test_data = MNIST(:test)

x_train = Float32.(reshape(train_data.features, 28 * 28, :)) 
y_train = Flux.onehotbatch(train_data.targets, 0:9)
x_test = Float32.(reshape(test_data.features, 28 * 28, :))  
y_test = Flux.onehotbatch(test_data.targets, 0:9)

rnn_settings = (
    input = UInt32(14*14),
    hidden = UInt32(64),
    output = UInt32(10),
)

settings = (
    eta = Float32(15e-3),
    epochs = UInt32(5),
    batch_size = UInt32(100),
)

wxh = Variable(xavier_init( rnn_settings.hidden,rnn_settings.input)) 
dense1 = Variable(xavier_init( rnn_settings.output,rnn_settings.hidden))
whh=Variable(zeros(Float32,rnn_settings.hidden,rnn_settings.hidden))
b = Variable(zeros(Float32, rnn_settings.hidden))
net = RNN_(wxh,whh,b, dense1)

train(net, x_train, y_train, settings,rnn_settings)
test(net, x_test, y_test,rnn_settings)

Epoch: 1
Train accuracy: 0.6426
Epoch: 2
Train accuracy: 0.8283
Epoch: 3
Train accuracy: 0.8718
Epoch: 4
Train accuracy: 0.8911
Epoch: 5
Train accuracy: 0.9021
135.051066 seconds (176.78 M allocations: 84.490 GiB, 29.06% gc time, 0.61% compilation time)
Test accuracy: 0.9098

